In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import simple_soil
import flopy.plot.styles as styles

In [ ]:
cv = simple_soil.base.ControlVolume(smoothing_omega=0.075)

In [ ]:
cv

In [ ]:
dtheta = 0.0005
soil_moisture = np.arange(0, cv.theta_sat + 10.0 * dtheta, dtheta)

In [ ]:
soil_moisture

In [ ]:
fraction_saturation = simple_soil.utils.saturation_fraction(
    soil_moisture,
    cv.theta_sat,
    cv.smoothing_omega,
)

In [ ]:
fraction_gw = simple_soil.utils.groundwater_recharge_fraction(
    soil_moisture,
    cv.theta_sat,
    cv.theta_fc,
    smoothing_omega=cv.smoothing_omega,
)

In [ ]:
fraction_lateral = simple_soil.utils.lateral_discharge_fraction(
    soil_moisture,
    cv.theta_sat,
    cv.theta_fc,
    smoothing_omega=cv.smoothing_omega,
)

In [ ]:
fraction_surface = simple_soil.utils.surface_discharge_fraction(
    soil_moisture,
    cv.theta_sat,
    cv.theta_discharge,
    smoothing_omega=cv.smoothing_omega,
)

In [ ]:
fraction_infiltration = simple_soil.utils.surface_infiltration_fraction(
    soil_moisture,
    cv.theta_sat,
    cv.theta_discharge,
    smoothing_omega=cv.smoothing_omega,
)

In [ ]:
fraction_pet = simple_soil.utils.pet_fraction(
    soil_moisture,
    cv.theta_pet_max,
    cv.theta_wp,
    smoothing_omega=cv.smoothing_omega,
)

In [ ]:
ranges = (
    0,
    cv.theta_wp,
    cv.theta_pet_max,
    cv.theta_fc,
    cv.theta_discharge,
    cv.theta_sat,
    soil_moisture[-1],
)
colors = ("0.25", "brown", "green", "cyan", "blue", "none")
labels = (
    r"$< \theta_{wp}$",
    r"$\theta_{wp}$",
    r"$\theta_{pet}$",
    r"$\theta_{fc}$",
    r"$\theta_{sur}$",
    r"$\theta_{sat}$",
    None,
)

In [ ]:
line_dict = {"lw": 1.5, "color": "black", "clip_on": False}
moisture_dict = {"lw": 0.5, "color": "0.5", "ls": "--"}

In [ ]:
mosaic = """
    AB
    CD
    EF
    """
with styles.USGSPlot():
    fig = plt.figure(layout="constrained", figsize=(6, 8))
    ax_dict = fig.subplot_mosaic(mosaic, sharex=True, sharey=True)

    for tag in ax_dict.keys():
        ax = ax_dict[tag]
        for idx, xx in enumerate(ranges[1:-1]):
            ax.axvline(xx, **moisture_dict)
            styles.add_text(
                ax=ax,
                x=xx,
                y=1.01,
                transform=False,
                text=labels[idx + 1],
                ha="center",
                va="bottom",
            )
        ax.set_xlim(0, soil_moisture[-1])
        ax.set_ylim(0, 1)

    ax = ax_dict["A"]
    ax.plot(soil_moisture, fraction_infiltration, **line_dict)
    ax.set_ylabel("Infiltration Area Fraction")

    ax = ax_dict["B"]
    ax.plot(soil_moisture, fraction_pet, **line_dict)
    ax.set_ylabel("Potential Evapotranspiration Fraction")

    ax = ax_dict["C"]
    ax.plot(soil_moisture, fraction_gw, **line_dict)
    ax.set_ylabel("Recharge Discharge Fraction")

    ax = ax_dict["D"]
    ax.plot(soil_moisture, fraction_lateral, **line_dict)
    ax.set_ylabel("Lateral Discharge Fraction")

    ax = ax_dict["E"]
    ax.plot(soil_moisture, fraction_surface, **line_dict)
    ax.set_xlabel("Soil Moisture")
    ax.set_ylabel("Surface Discharge Area Fraction")

    ax = ax_dict["F"]
    ax.plot(soil_moisture, fraction_saturation, **line_dict)
    ax.set_xlabel("Soil moisture")
    ax.set_ylabel("Volume Change Fraction");